Atmospheres & Passbands
============================

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.1 installed. (You can comment out this line if you don't use pip for your installation or don't want to update to the latest release).

In [ ]:
!pip install -I "phoebe>=2.1,<2.2"

As always, let's do imports and initialize a logger and a new bundle.  See [Building a System](building_a_system.ipynb) for more details.

In [1]:
%matplotlib inline

In [2]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()

And we'll add a single light curve dataset to expose all the passband-dependent options.

In [3]:
b.add_dataset('lc', times=np.linspace(0,1,101), dataset='lc01')

<ParameterSet: 15 parameters | contexts: compute, dataset>

Relevant Parameters
-----------------------

An 'atm' parameter exists for each of the components in the system (for each set of compute options) and defines which atmosphere table should be used.

By default, these are set to 'ck2004' (Castelli-Kurucz) but can be set to 'blackbody' as well as 'extern_atmx' and 'extern_planckint' (which are included primarily for direct comparison with PHOEBE legacy).

In [4]:
b['atm']

<ParameterSet: 2 parameters | components: primary, secondary>

In [5]:
b['atm@primary']

<Parameter: atm=ck2004 | keys: description, choices, value, visible_if, copy_for>

In [6]:
b['atm@primary'].description

'Atmosphere table'

In [7]:
b['atm@primary'].choices

[u'blackbody', u'extern_atmx', u'extern_planckint', u'ck2004']

Note that if you change the value of 'atm' to anything other than 'ck2004', the corresponding 'ld_func' will need to be changed to something other than 'interp' (warnings and errors will be raised to remind you of this).

In [8]:
b['ld_func@primary']

<Parameter: ld_func=interp | keys: description, choices, value, visible_if, copy_for>

In [9]:
b['atm@primary'] = 'blackbody'

Wed, 30 Jan 2019 16:25 PARAMETERS   WARNING ld_func='interp' only supported by atm='ck2004'.  Either change atm@primary or ld_func@primary@lc01  If not addressed, this warning will continue to be raised and will throw an error at run_compute.


In [10]:
print(b.run_checks())

(False, "ld_func='interp' only supported by atm='ck2004'.  Either change atm@primary or ld_func@primary@lc01")


In [11]:
b['ld_func@primary'] = 'logarithmic'

In [12]:
print(b.run_checks())

(True, '')


A 'passband' parameter exists for each passband-dependent-dataset (i.e. not meshes or orbits, but light curves and radial velocities).  This parameter dictates which passband should be used for the computation of all intensities.

In [13]:
b['passband']

<Parameter: passband=Johnson:V | keys: description, choices, value, visible_if, copy_for>

The available choices will include both locally installed passbands as well as passbands currently available from the online PHOEBE repository.  If you choose an online-passband, it will be downloaded and installed locally as soon as required by [b.run_compute](../api/phoebe.frontend.bundle.Bundle.run_compute.md).

In [14]:
print(b['passband'].choices)

['STEREO:HI1A-noQE', 'Cousins:R', 'Stromgren:b', 'Brite:B', 'STEREO:HI1B-QE', 'Stromgren:v', 'Hipparcos:Hp', 'Stromgren:u', 'Brite:R', 'Stromgren:y', 'Johnson:B', 'Gaia:RP', 'STEREO:HI1B-noQE', 'Tycho:VT', 'Bolometric:900-40000', 'Johnson:R', 'Johnson:V', 'Johnson:U', 'Gaia:RVS', 'Kepler:mean', 'Gaia:G', 'Tycho:BT', 'Spitzer:5.8um', 'Gaia:BP', 'Spitzer:4.5um', 'KELT:R', 'LSST:i', 'LSST:g', 'TESS:default', 'Spitzer:3.6um', 'Cousins:I', 'LSST:z', 'STEREO:HI1A-QE', 'LSST:y', 'LSST:u', 'LSST:r', 'Spitzer:8.0um']


To see your current locally-installed passbands, call [phoebe.list_installed_passbands()](../api/phoebe.list_installed_passbands.md).

In [15]:
print(phoebe.list_installed_passbands())

['Johnson:V']


These installed passbands can be in any of a number of directories, which can be accessed via [phoebe.list_passband_directories()](../api/phoebe.list_passband_directories.md).

The first entry is the global location - this is where passbands can be stored by a server-admin to be available to all PHOEBE-users on that machine.

The second entry is the local location - this is where individual users can store passbands and where PHOEBE will download and install passbands (by default).

In [16]:
print(phoebe.list_passband_directories())

('/home/kyle/.local/lib/python2.7/site-packages/phoebe/atmospheres/tables/passbands/',
 '/home/kyle/.phoebe/atmospheres/tables/passbands/')

To see the passbands available from the [online repository](http://github.com/phoebe-project/phoebe2-tables), call [phoebe.list_online_passbands()](../api/phoebe.list_online_passbands.md).

In [18]:
print(phoebe.list_online_passbands())

['STEREO:HI1A-noQE', 'Spitzer:3.6um', 'Stromgren:b', 'Brite:B', 'STEREO:HI1B-QE', 'Stromgren:v', 'Hipparcos:Hp', 'Stromgren:u', 'Brite:R', 'Stromgren:y', 'Johnson:B', 'Gaia:RP', 'STEREO:HI1B-noQE', 'Tycho:VT', 'Bolometric:900-40000', 'Johnson:R', 'Johnson:V', 'Johnson:U', 'Gaia:RVS', 'Kepler:mean', 'LSST:i', 'Gaia:G', 'Tycho:BT', 'Spitzer:5.8um', 'Spitzer:4.5um', 'KELT:R', 'Gaia:BP', 'LSST:g', 'TESS:default', 'Cousins:R', 'Cousins:I', 'LSST:z', 'STEREO:HI1A-QE', 'LSST:y', 'LSST:u', 'LSST:r', 'Spitzer:8.0um']


Lastly, to manually download and install one of these online passbands, you can do so explicitly via [phoebe.download_passband](../api/phoebe.download_passband.md).

Note that this isn't necessary unless you want to explicitly download passbands before needed by run_compute (perhaps if you're expecting to have unreliable network connection in the future and want to ensure you have all needed passbands).

In [19]:
phoebe.download_passband('Cousins:R')

In [20]:
print(phoebe.list_installed_passbands())

['Cousins:R', 'Johnson:V']
